#  1. SQL запросы для получения данных

1. Найдите количество рейсов на каждой модели самолёта с вылетом в сентябре 2018 года. Назовите получившееся поле flights_amount и выведите его. Также напечатайте на экране поле model.

``` SQL
SELECT
    aircrafts.model AS model,
    COUNT(flights.aircraft_code) AS flights_amount
FROM
    flights
INNER JOIN aircrafts ON aircrafts.aircraft_code = flights.aircraft_code
WHERE
    departure_time::date BETWEEN '2018-09-01' AND '2018-09-30'
GROUP BY
    model
```

2. Посчитайте количество отправленных рейсов по всем моделям самолётов Boeing и Airbus в сентябре. Назовите получившуюся переменную flights_amount и выведите её.

``` SQL
SELECT
    COUNT(flights.aircraft_code) AS flights_amount
FROM
    flights
INNER JOIN aircrafts ON aircrafts.aircraft_code = flights.aircraft_code
WHERE 
    (model LIKE 'Boeing%' OR model LIKE 'Airbus%')
     AND EXTRACT(month from flights.departure_time :: date) = '09'
```

3. Посчитайте среднее количество прибывающих рейсов в день для каждого города за август 2018 года. Назовите получившееся поле average_flights, вместе с ним выведите столбец city.

``` SQL
SELECT
    flights_august.city AS city,
    AVG(flights_august.count_flights) AS average_flights
FROM
    (SELECT
        COUNT(flights.flight_id) AS count_flights,
        airports.city AS city,
        flights.arrival_time::date AS date
    FROM
        airports
    INNER JOIN flights ON airports.airport_code = flights.arrival_airport
    WHERE
        arrival_time::date BETWEEN '2018-08-01' AND '2018-08-31'
    GROUP BY
        city,
        date) 
            AS flights_august
GROUP BY
    city
```

4. Установите фестивали, которые проходили с 23 июля по 30 сентября 2018 года в Москве, и номер недели, в которую они проходили. Выведите название фестиваля festival_name и номер недели festival_week.

``` SQL
SELECT
    festival_name,
    EXTRACT('week' FROM festival_date) AS festival_week
FROM
    festivals
WHERE
    festival_date BETWEEN '2018-07-23' AND '2018-09-30' AND festival_city = 'Москва'
```

5. Для каждой недели с 23 июля по 30 сентября 2018 года посчитайте количество приобретённых билетов в Москву. 

``` SQL
SELECT 
	T.week_number,
	T.ticket_amount,
	T.festival_week,
	T.festival_name
FROM 
(
(
    SELECT
        EXTRACT('week' FROM flights.arrival_time) AS week_number,
        COUNT(ticket_flights.ticket_no) AS ticket_amount       
    FROM
       flights
    INNER JOIN ticket_flights ON ticket_flights.flight_id= flights.flight_id
    INNER JOIN airports ON airports.airport_code = flights.arrival_airport
    WHERE
        flights.departure_time::date BETWEEN '2018-07-23' AND '2018-09-30' 
        AND  airports.city = 'Москва'
    GRoup BY
        week_number
) t
LEFT JOIN 
(SELECT 		
		festival_name,	
		EXTRACT (week FROM festivals.festival_date) AS festival_week
	FROM 
		festivals
	WHERE
		festival_city = 'Москва'
	  AND CAST(festivals.festival_date AS date) BETWEEN '2018-07-23' AND '2018-09-30'
) t2 
ON 
	t.week_number = t2.festival_week
) AS T;
```

**Аналитика средствами Python**

У вас есть файлы, в которых содержатся результаты запросов из предыдущих заданий.

**query_1.csv** — результат первого запроса. В нём содержится такая информация:

- *model* — **модель самолета;
- *flights_amount* — количество рейсов для каждой модели самолетов *model* в сентябре 2018 года.

**query_3.csv** — результат третьего запроса. В нём содержится такая информация:

- *city* — город;
- *average_flights* — среднее количество рейсов, прибывающих в город (*city*) за день в сентябре 2018 года.

 Для этих двух наборов данных нужно:

- импортировать файлы;
- изучить данные в них;
- проверить типы данных на корректность;
- выбрать топ-10 городов по количеству рейсов;
- построить графики: модели самолётов и количество рейсов, города и количество рейсов, топ-10 городов и количество рейсов;
- сделать выводы по каждому из графиков, пояснить результат.

**Шаг 2.  Проверка гипотезы средствами Python**

**query_last.csv** — результат последнего запроса. В нём следующая информация:

- *week_number* — **номер недели;
- *ticket_amount* — количество проданных билетов за неделю;
- *festival_week —* есть ли на этой неделе фестиваль;
- *festival_name —* название фестиваля.

Проверьте гипотезу: «Средний спрос на билеты во время фестивалей не отличается от среднего спроса на билеты в обычное время».

Пороговое значение *alpha* задайте самостоятельно.

Поясните:

- Как вы формулировали нулевую и альтернативную гипотезы;
- Какой критерий использовали для проверки гипотез и почему.

# 2. Аналитика

pip install plotly==4.8.1 --user

pip install chart_studio

pip install cufflinks plotly --user

In [1]:
#ипортируем необходимые библиотеки
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st

# Стандартные импорты plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()
import plotly.express as px
# Устанавливаем глобальную тему 
cf.set_config_file(world_readable=True, theme='pearl', offline=True)

In [2]:
#импортируем датасеты
model_flights = pd.read_csv('/home/andrey/Datasets/query_1.csv')
city_average = pd.read_csv('/home/andrey/Datasets/query_3.csv')
last = pd.read_csv('//home/andrey/Datasets/query_last.csv')

In [3]:
#изучим данные
print(model_flights.info())
print('_____________________')
print(model_flights.describe())
model_flights.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   model           8 non-null      object
 1   flights_amount  8 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 256.0+ bytes
None
_____________________
       flights_amount
count        8.000000
mean      2035.625000
std       1965.203947
min        300.000000
25%        605.250000
50%        795.000000
75%       4250.250000
max       4557.000000


,model,flights_amount
0,Airbus A319-100,607
1,Airbus A321-200,960
2,Boeing 737-300,630
3,Boeing 767-300,600
4,Boeing 777-300,300
5,Bombardier CRJ-200,4446
6,Cessna 208 Caravan,4557
7,Sukhoi SuperJet-100,4185


In [4]:
print(city_average.info())
print('_____________________')
print(city_average.describe())
city_average.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city             101 non-null    object 
 1   average_flights  101 non-null    float64
dtypes: float64(1), object(1)
memory usage: 1.7+ KB
None
_____________________
       average_flights
count       101.000000
mean          5.494189
std          13.119790
min           1.000000
25%           2.000000
50%           3.000000
75%           5.870968
max         129.774194


,city,average_flights
0,Абакан,3.870968
1,Анадырь,1.000000
2,Анапа,2.161290
3,Архангельск,5.354839
4,Астрахань,2.451613


Есть 8 записей по моделям самолета, и 101 запись с городами, данные выглядят корректно. При этом, среднее количество полётов каждой модели достаточно сильно отличается.

In [5]:
print(last.info())
print('_____________________')
print(last.describe())
last.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   week_number    10 non-null     int64  
 1   ticket_amount  10 non-null     int64  
 2   festival_week  3 non-null      float64
 3   festival_name  3 non-null      object 
dtypes: float64(1), int64(2), object(1)
memory usage: 448.0+ bytes
None
_____________________
       week_number  ticket_amount  festival_week
count     10.00000      10.000000       3.000000
mean      34.50000   50670.400000      32.333333
std        3.02765    2502.752716       3.214550
min       30.00000   43568.000000      30.000000
25%       32.25000   51364.500000      30.500000
50%       34.50000   51439.000000      31.000000
75%       36.75000   51596.750000      33.500000
max       39.00000   51675.000000      36.000000


,week_number,ticket_amount,festival_week,festival_name
0,30,43568,30.0,Park Live
1,31,51034,31.0,Пикник Афиши
2,32,51675,NaN,NaN
3,33,51378,NaN,NaN
4,34,51492,NaN,NaN


Только в третьем таблице (last) есть некорректный тип данных, в столбце festival_week тип данных указан как float64 заменим его на int64 пропущенные значения заменим на нули. 

In [6]:
last['festival_week'] = last['festival_week'].fillna(0).astype('int')

In [7]:
#найдём топ 10 городов по количеству полётов.
top_10_city = city_average.sort_values(by = "average_flights", ascending=False).head(10)
top_10_city

,city,average_flights
43,Москва,129.774194
70,Санкт-Петербург,31.161290
54,Новосибирск,17.322581
33,Красноярск,11.580645
20,Екатеринбург,11.322581
67,Ростов-на-Дону,10.193548
63,Пермь,10.129032
10,Брянск,10.000000
74,Сочи,9.612903
84,Ульяновск,9.580645


Весьма логично. что в топ вошли самые крупные Российские города. При этом Москва многократно опережает другие города.

In [8]:
#построим график модели самолетов и количество рейсов
model_flights
fig = px.bar(model_flights, x='model', y='flights_amount',
             title="Модель самолёта и количество совершенных рейсов",
              color='model',
              height=400)
fig.show()

Из полученного выше графика видно, что большинство рейсов совершается на "маленьких" самолётах, что скорее всего связано с короткими региональными перелётами, которые выполняются по несколько раз в день одним самолётом и общим большим количеством таких самолётов, когда в свою очередь более крупные и дорогие Boeing и Airbus совершают в день максимум 1-2 перелёта.

In [9]:
#Построим график города и количество рейсов
model_flights
fig = px.bar(city_average, x='city', y='average_flights',
             title="Город и количество рейсов",
              color='city',
              height=400)
fig.show()

Из полученного графика видно, что большинство рейсов прибывает в Москву и Санкт-Петербург, т.к. эти города являются крупными авиационными хабами, и довольно часто возникает ситуация когда между соседними городами нет прямого авиасообщения и пассажиры вынуждены лететь через Москву или Санкт-Петербург.

In [10]:
#Построим график топ-10 городов и количества рейсов
model_flights
fig = px.bar(top_10_city, x='city', y='average_flights',
             title="ТОП 10 городов по количеству прибывающих рейсов",
              color='city',
              height=400)
fig.show()

Из полученного графика видно, что чем крупнее город и чем более транспортно важным он является, тем большее количество рейсов в него прибывает.

# 3. Проверка гипотезы

## Задача
Средний спрос на билеты во время фестивалей не отличается от среднего спроса на билеты в обычное время

Составим гипотезы:

##### Н0 - спрос на билеты во время фестивалей не отличается от среднего спроса в обычные дни
##### Н1 - спрос на билеты во время фестивалей отличается от среднего спроса в обычные дни

Для решения задачи возьмём две выборки, первую с пассажирами в недели проведения фестивалей, вторую когда фестивали не проводились. Т.к. мы проверяем средние значения у двух независимых выборок потребуется критерий стьюдента.

Пороговое значение установим равное 0.05

In [11]:
festival = last[last['festival_week'] >0]['ticket_amount']
non_festival = last[last['festival_week'] == 0]['ticket_amount']

In [12]:
alpha = .05

result = st.ttest_ind(festival,non_festival, equal_var = False)

if result.pvalue >= alpha:
    print("P-val =", round(result.pvalue, 5),"\nАльтернативная гипотеза отклонена. Скорее всего средний спрос не отличается.")
else:
    print("P-val =", round(result.pvalue, 5), "\Нулевая гипотеза отклонена. Скорее всего средний спрос отличается.")

P-val = 0.37743 
Альтернативная гипотеза отклонена. Скорее всего средний спрос не отличается.


## Общий вывод

В результате анализа полученных данных можно сделать следующие выводы:
- Большинство рейсов выполняется малыми самолётами для осуществления локальных перелётов.
- Абсолютное большинство рейсов осуществляется в Москву.
- Средний спрос на билеты во время проведения фестивалей не отличается от обычного.